In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.2.0
sys.version_info(major=3, minor=6, micro=9, releaselevel='final', serial=0)
matplotlib 3.3.4
numpy 1.19.5
pandas 1.1.5
sklearn 0.24.2
tensorflow 2.2.0
tensorflow.keras 2.3.0-tf


# 把py的函数变为图是为了提升效率
Python 本身函数的执行效率不高，
如果 python 函数能够像 tf 的库函数来使用是不是很好，因此 tf 针对编译器和各种设备做了各种优化

In [4]:
# tf.function and auto-graph.
#自己实现一下selu激活函数,如果scale不为1，那就是selu
def scaled_elu(z, scale=1.0, alpha=1.0):
    # z >= 0 ? scale * z : scale * alpha * tf.nn.elu(z)
    # 是不是大于等于0
    is_positive = tf.greater_equal(z, 0.0)
#     return scale * tf.where(is_positive, z, alpha * tf.nn.elu(z))
    return scale * tf.where(is_positive, z, alpha * (tf.math.exp(z)-1))

#运行一下，这还是py函数
print(scaled_elu(tf.constant(-3.)))
print(scaled_elu(tf.constant([-3., -2.5])))

#把py实现的函数变为图实现的函数
#scaled_elu_tf就是图
scaled_elu_tf = tf.function(scaled_elu)
print(scaled_elu_tf(tf.constant(-3.)))
print(scaled_elu_tf(tf.constant([-3., -2.5])))

#可以通过这种方式（python_function）找回原来的py函数
# is是判断，查一下字符串判断is和==的区别
print(scaled_elu_tf.python_function is scaled_elu)
print(scaled_elu_tf.python_function)
print(scaled_elu)
print(scaled_elu_tf)#tf的函数的执行效率比较高

tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
tf.Tensor(-0.95021296, shape=(), dtype=float32)
tf.Tensor([-0.95021296 -0.917915  ], shape=(2,), dtype=float32)
True
<function scaled_elu at 0x7f34e67f2d08>
<function scaled_elu at 0x7f34e67f2d08>


In [ ]:
#我们来测试一下性能，100万个数
%timeit scaled_elu(tf.random.normal((10000, 10000)))
%timeit scaled_elu_tf(tf.random.normal((10000, 10000)))

In [ ]:
# 1 + 1/2 + 1/2^2 + ... + 1/2^n
#加了@tf.function装饰后就变为图结果，但是输入类型上不会有变化
@tf.function
def converge_to_2(n_iters):
    total = tf.constant(0.)
    increment = tf.constant(1.)
    for _ in range(n_iters):
        total += increment
        increment /= 2.0
    return total
print(converge_to_2)
print(converge_to_2(20))

In [ ]:
#如何看tf的图的代码
def display_tf_code(func):
    code = tf.autograph.to_code(func)
    from IPython.display import display, Markdown
    display(Markdown('```python\n{}\n```'.format(code)))

In [ ]:
#传普通py函数,返回的是tf图的代码
display_tf_code(scaled_elu)

In [ ]:
#这个的前提是去除converge_to_2的装饰
# 因为converge_to_2有@tf.function标注，去掉应该就没问题了。to_code函数的输入是module,
# class, method, function, traceback, frame, or code object。不能是tf function.
# 传入display_tf_code的必须是python代码，已经是图的传进去会报错
display_tf_code(converge_to_2.python_function)

In [ ]:
#tf要把变量定义在函数外面，不能放里边
var = tf.Variable(0.)

@tf.function
def add_21():
    return var.assign_add(21) # += 

print(add_21())

In [ ]:
#cube计算立方，py是泛型设计，我们通过input_signature加类型限制可以防止调错
# 这里就设置必须传入整形
# @tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
@tf.function
def cube(z):
    return tf.pow(z, 3)

try:
    print(cube(tf.constant([1., 2., 3.])))
except ValueError as ex:
    print(ex)

print('-'*50)
#这行没问题
print(cube(tf.constant([1, 2, 3])))
print(cube)

In [ ]:
int('abc')

In [ ]:
# @tf.function py func -> tf graph
# get_concrete_function -> 给tf.function add input signature -> SavedModel
# 这里是额外增加了一个处理，即必须传入整形
# 有了这句就不能有 @tf.function(input_signature=[tf.TensorSpec([None], tf.int32, name='x')])
cube_func_int32 = cube.get_concrete_function(
    tf.TensorSpec([None], tf.int32))
print(cube_func_int32)
print(cube)

try:
    print(cube_func_int32(tf.constant([1, 2, 3])))
except Exception as ex:
    print(ex)

In [ ]:
# 没必要这样做
#我们只要看原来函数和新生成的是否一致
# print(cube_func_int32 is cube.get_concrete_function())
print(cube.get_concrete_function(
    tf.constant([1, 2, 3])))
print(cube_func_int32)
print(cube_func_int32 is cube.get_concrete_function(
    tf.constant([1, 2, 3])))

In [ ]:
print(cube_func_int32)
cube_func_int32.graph

In [ ]:
#看下图定义都有哪些操作,了解即可
cube_func_int32.graph.get_operations()

In [ ]:
pow_op = cube_func_int32.graph.get_operations()[0]
print(pow_op)

In [ ]:
print(list(pow_op.inputs))
print('-'*50)
print(list(pow_op.outputs))


In [ ]:
#Placeholder用来放输入的地方，2.0中不需要，图中依然保留了
cube_func_int32.graph.get_operation_by_name("z")

In [ ]:
cube_func_int32.graph.get_tensor_by_name("z:0")

In [ ]:
#打印出来看看图信息
cube_func_int32.graph.as_graph_def()